## Output the constraints on DA/rd and H(z)rd from spec-z, photo-z, spec-z+cross, spec-z+photo-z+cross in a table. 
Update the result with Sigma_fog/(1+z). --03-28-2023

In [1]:
import os, sys
import numpy as np
from pathlib import Path
sys.path.append("/home/zjding/csst_bao/fisher_pkmu/")
from mcmc_funs import growth_factor
from astropy.table import Table

In [2]:
speed_c = 299792.458    # speed of light, km/s
Omega_m = 0.3075

survey_area = 17500.0
kmax = 0.3
zmin = 0.
zmax = 1.6
nzbins = 8
zbins = np.linspace(zmin, zmax, nzbins+1)

const_low = 0.99
const_up = 1.01

kwidth = 0.005

Pspecz_sys = "1.0"

skyarea_total = 4*np.pi * (180./np.pi)**2.0
fsky = survey_area/skyarea_total
G_0 = growth_factor(0.0, Omega_m)       # G_0 at z=0, normalization factor 

# ## estimate the power spectrum damping parameter $\Sigma_{specz}$, $\Sigma_{pz}$ from the spec-z and photo-z uncertainty
sigma_specz = 0.002
sigma_photoz = 0.025     # does not matter here if we only care about the spec-z result
Sigma_fog = 7.0          # Finger-of-God damping term

tracer_list = ['specz', 'photoz', 'specz_add_cross', 'specz_photoz_add_cross']
if Sigma_fog > 0.0:
    params_str_list = ['11110101010101', '11101010101010', '11111111111111', '11111111111111']

#idir0 = "/home/zjding/csst_bao/fisher_pkmu/BAO_part/Fisher_sigma_alphas/numerical_method/secondgen/output/"
#alpha_dir0 = f"{idir0}/sigma_alpha/vary_params_{const_low}_{const_up}/Sigma_fog_{Sigma_fog:.1f}/%s/k_width{kwidth}/Pspecz_sys{Pspecz_sys}/params_bf1.0/"

idir0 = "/home/zjding/csst_bao/fisher_pkmu/BAO_part/Fisher_sigma_alphas/numerical_method/default/output/"
alpha_dir0 = f"{idir0}/sigma_alpha/vary_params_{const_low}_{const_up}/Sigma_fog_{Sigma_fog:.1f}/%s/k_width{kwidth}/Pspecz_sys{Pspecz_sys}/params_bf1.0/"   # for Psys=0
recon_dir_list = ['pre_recon', 'post_recon']

## collect data

In [3]:
def read_data(alpha_dir, params_str, sigma_alperp_list, sigma_alpara_list):
    filename = f"sigma_alpha_{tracer}_tracer_zerror_specz{sigma_specz:.3f}_photoz{sigma_photoz:.3f}_kmax{kmax:.2f}_params{params_str}.npz"

    ifile = Path(alpha_dir, filename)
    data = np.load(ifile)
    sigma_alphas_mar = data['alphas_mar']

    sigma_alperp_list.append(sigma_alphas_mar[:, 2] * 100)  # transfer to per cent
    sigma_alpara_list.append(sigma_alphas_mar[:, 3] * 100)
    return sigma_alperp_list, sigma_alpara_list

In [4]:
sigma_alperp_list = []
sigma_alpara_list = []

for tracer, params_str in zip(tracer_list, params_str_list):
    if tracer == 'photoz':
        recon_dir = recon_dir_list[0]
        ##alpha_dir = alpha_dir0.format(idir0, const_low, const_up, zmin, zmax, nzbins, Sigma_fog, recon_dir, kwidth, Pspecz_sys)
        alpha_dir = alpha_dir0%(recon_dir)
        sigma_alperp_list, sigma_alpara_list = read_data(alpha_dir, params_str, sigma_alperp_list, sigma_alpara_list)
    else:
        for recon_dir in recon_dir_list:
            alpha_dir = alpha_dir0%(recon_dir)
            sigma_alperp_list, sigma_alpara_list = read_data(alpha_dir, params_str, sigma_alperp_list, sigma_alpara_list)
        
sigma_alperp_array = np.array(sigma_alperp_list).T    # transpose it to make the row as the redshift index
sigma_alpara_array = np.array(sigma_alpara_list).T

In [5]:
sigma_alphas_array = np.hstack((sigma_alperp_array, sigma_alpara_array))
print(sigma_alphas_array.shape)

(8, 14)


In [6]:
data = [zbins[0:-1], zbins[1:]] + sigma_alperp_list + sigma_alpara_list
data

[array([0. , 0.2, 0.4, 0.6, 0.8, 1. , 1.2, 1.4]),
 array([0.2, 0.4, 0.6, 0.8, 1. , 1.2, 1.4, 1.6]),
 array([ 4.74066766,  1.74130739,  1.12031334,  0.84110686,  0.77040378,
         1.4121305 ,  5.04690568, 24.51901026]),
 array([ 2.25977766,  0.90957142,  0.6539332 ,  0.53060696,  0.53094793,
         1.12112372,  4.58883556, 24.51901026]),
 array([6.57809909, 2.5466111 , 1.6355923 , 1.2178468 , 1.00168856,
        0.89837463, 0.86578434, 0.82619238]),
 array([4.7147418 , 1.72429954, 1.08433883, 0.80396137, 0.70100081,
        0.93140478, 1.78146575, 3.92486628]),
 array([1.32293079, 0.72178706, 0.6381136 , 0.52923005, 0.52093464,
        0.80933632, 1.69251593, 3.92486628]),
 array([4.71454716, 1.72405683, 1.08264196, 0.80150294, 0.69207226,
        0.76766783, 0.81008408, 0.80865188]),
 array([0.91530916, 0.65649117, 0.63724327, 0.52920851, 0.52066841,
        0.7147838 , 0.80583728, 0.80865188]),
 array([12.20566884,  4.64538284,  3.00514817,  2.23828116,  1.98527603,
         3.14

In [7]:
len(data)

16

In [8]:
name_list = ['zmin', 'zmax', 'alperp_specz_pre', 'alperp_specz_post', 'alperp_photoz', 'alperp_specz_pre+cross', 'alperp_specz_post+cross', 'alperp_joint_pre', 'alperp_join_post', \
                             'alpara_specz_pre', 'alpara_specz_post', 'alpara_photoz', 'alpara_specz_pre+cross', 'alpara_specz_post+cross', 'alpara_joint_pre', 'alpara_join_post']
print(len(name_list))
res = Table(data, names=name_list)
res

16


zmin,zmax,alperp_specz_pre,alperp_specz_post,alperp_photoz,alperp_specz_pre+cross,alperp_specz_post+cross,alperp_joint_pre,alperp_join_post,alpara_specz_pre,alpara_specz_post,alpara_photoz,alpara_specz_pre+cross,alpara_specz_post+cross,alpara_joint_pre,alpara_join_post
float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
0.0,0.2,4.740667660922299,2.25977766315452,6.578099085936952,4.714741801406799,1.3229307865048423,4.714547156892048,0.9153091648364117,12.205668842398593,4.838721665585517,90.48368473279635,12.195569302527671,4.573857733366469,12.19516846470568,4.492756677940939
0.2,0.4,1.7413073871497666,0.9095714207982849,2.5466110956706616,1.7242995354530575,0.7217870588632186,1.7240568303712636,0.6564911736098057,4.64538284222831,1.9975600049637716,46.03623896859525,4.638494371489535,1.9365280639563613,4.638286669434908,1.9174789298372887
0.4,0.6000000000000001,1.1203133397628309,0.6539332041528182,1.635592302816428,1.0843388317873983,0.6381136038893218,1.0826419605246507,0.6372432672897984,3.0051481713124226,1.4849109974993966,37.35176648560501,2.9907711423149723,1.4791847431790037,2.9900094573096263,1.4788072003162482
0.6000000000000001,0.8,0.8411068599073335,0.530606958132633,1.2178468044406934,0.8039613672641549,0.5292300467460263,0.8015029415031504,0.5292085141337184,2.2382811584752096,1.199796020353226,30.841990643454366,2.2236330354784735,1.1992576863428523,2.2226041199488864,1.1992020020066476
0.8,1.0,0.7704037796699718,0.5309479334471272,1.001688556029953,0.7010008062637106,0.5209346394356651,0.6920722604394469,0.5206684110974122,1.9852760292443452,1.191262420193396,27.547940056715152,1.9596837476062228,1.187836690999558,1.9563905007816356,1.1877193599430318
1.0,1.2000000000000002,1.412130503268734,1.1211237157997802,0.8983746253248468,0.931404783599669,0.809336320982282,0.7676678324742443,0.7147838025234406,3.140189530900159,2.296031639775564,27.382738452116378,3.007433064152695,2.2135149629880853,2.969339161075215,2.1916538112075115
1.2000000000000002,1.4000000000000001,5.046905679424804,4.588835558155407,0.8657843379162187,1.781465749785693,1.6925159268037293,0.8100840821686619,0.8058372798386834,9.449649078676712,8.334199525029453,29.31094840271974,8.648963886780964,7.667304480170399,8.372029587045004,7.454695860297539
1.4000000000000001,1.6,24.51901025965155,24.51901025965155,0.8261923753444119,3.9248662798101233,3.9248662798101233,0.8086518752548266,0.8086518752548266,42.68977253080505,42.68977253080505,28.868458668757533,33.60929198877235,33.60929198877235,22.875511910817796,22.875511910817796


In [9]:
ncol_alphas = sigma_alphas_array.shape[1]//2

In [10]:
odir = "./Table/"
if not os.path.exists(odir):
    os.makedirs(odir)

In [11]:
header=" "
for i in name_list:
    header += i + ", "

ofile = odir + "sigma_DA_Hz_Sigmafog_{0:.1f}_Pspeczsys{1}_pre_postrecon.csv".format(Sigma_fog, Pspecz_sys)

with open(ofile, 'w') as fwriter:
    for i in range(nzbins):
        zmin, zmax = zbins[i], zbins[i+1]
        text = f"{zmin:.1f} < z < {zmax:.1f}"
        for j in range(2):
            text += f"& {sigma_alphas_array[i, 0+j*ncol_alphas]:.2f} ({sigma_alphas_array[i, 1+j*ncol_alphas]:.2f}) "
            text += f"& {sigma_alphas_array[i, 2+j*ncol_alphas]:.2f} " 
            text += f"& {sigma_alphas_array[i, 3+j*ncol_alphas]:.2f} ({sigma_alphas_array[i, 4+j*ncol_alphas]:.2f}) "
            text += f"& {sigma_alphas_array[i, 5+j*ncol_alphas]:.2f} ({sigma_alphas_array[i, 6+j*ncol_alphas]:.2f}) "
            if j == 0:
                text += "& "
        
        text += " \\\\\n"
        fwriter.write(text)

## only for pre-recon results

In [12]:
name_list_pre = ['zmin', 'zmax', 'alperp_specz_pre', 'alperp_photoz', 'alperp_specz_pre+cross', 'alperp_joint_pre', \
                             'alpara_specz_pre', 'alpara_photoz', 'alpara_specz_pre+cross', 'alpara_joint_pre']

ofile = odir + "sigma_DA_Hz_Sigmafog_{0:.1f}_Pspeczsys{1}_pre_recon.csv".format(Sigma_fog, Pspecz_sys)

header=" "
for i in name_list_pre:
    header += i + ", "

with open(ofile, 'w') as fwriter:
    for i in range(nzbins):
        zmin, zmax = zbins[i], zbins[i+1]
        text = f"{zmin:.1f} < z < {zmax:.1f}"
        for j in range(2):
            text += f"& {sigma_alphas_array[i, 0+j*ncol_alphas]:.2f} "
            text += f"& {sigma_alphas_array[i, 2+j*ncol_alphas]:.2f} " 
            text += f"& {sigma_alphas_array[i, 3+j*ncol_alphas]:.2f} "
            text += f"& {sigma_alphas_array[i, 5+j*ncol_alphas]:.2f} "
            if j == 0:
                text += "& "
        
        text += " \\\\\n"
        fwriter.write(text)